In [ ]:
import socket,warnings
try:
    socket.setdefaulttimeout(1)
    socket.socket(socket.AF_INET, socket.SOCK_STREAM).connect(('1.1.1.1', 53))
except socket.error as ex: raise Exception("STOP: No internet. Click '>|' in top right and set 'Internet' switch to on")

In [ ]:
import os
iskaggle = os.environ.get('KAGGLE_KERNEL_RUN_TYPE', '')

if iskaggle:
    !pip install -Uqq fastai duckduckgo_search 

In [ ]:
from duckduckgo_search import ddg_images
from fastcore.all import *

def search_images(term, max_images=30):
    print(f"Searching for '{term}'")
    return L(ddg_images(term, max_results=max_images)).itemgot('image')

In [ ]:
urls = search_images('itachi photos', max_images=1)
urls[0]

In [ ]:
from fastdownload import download_url
dest = 'itachi.jpg'
download_url(urls[0], dest, show_progress=False)

from fastai.vision.all import *
im = Image.open(dest)
im.to_thumb(256,256)

In [ ]:
download_url(search_images('sasuke photos', max_images=1)[0], 'sasuke.jpg', show_progress=False)
Image.open('sasuke.jpg').to_thumb(256,256)

In [ ]:
searches = 'itachi uchiha','sasuke uchiha',
path = Path('uchiha_classification')
from time import sleep

for o in searches:
    dest = (path/o)
    dest.mkdir(exist_ok=True, parents=True)
    download_images(dest, urls=search_images(f'{o} photos'))
    sleep(10)  # Pause between searches to avoid over-loading server
    download_images(dest, urls=search_images(f'{o} shippuden photos'))
    sleep(10)
    download_images(dest, urls=search_images(f'{o} shonen jump photos'))
    sleep(10)
    resize_images(path/o, max_size=400, dest=path/o)

In [ ]:
path.ls()

In [ ]:
failed = verify_images(get_image_files(path))
failed.map(Path.unlink)
len(failed)

In [ ]:
dls = DataBlock(
    blocks=(ImageBlock, CategoryBlock), 
    get_items=get_image_files, 
    splitter=RandomSplitter(valid_pct=0.2, seed=42),
    get_y=parent_label,
    item_tfms=[Resize(192, method='squish')]
).dataloaders(path, bs=32)

dls.show_batch(max_n=10)

In [ ]:
learn = vision_learner(dls, resnet18, metrics=error_rate)
learn.lr_find()

In [ ]:
learn.fine_tune(6, 1e-3)

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix()
interp.plot_top_losses(9, figsize=(15,10))

In [ ]:
from fastai.vision.widgets import *
cleaner = ImageClassifierCleaner(learn)
cleaner

In [ ]:
learn.fine_tune(2, 1e-3)

In [ ]:
learn.show_results()

In [ ]:
this_uchiha,num,probs = learn.predict(PILImage.create('sasuke.jpg'))
print(f"This is a photo of {this_uchiha}.")
if this_uchiha=='itachi uchiha':
    print(f"Probability it's a photo of itachi uchiha: {probs[0]}")
else:
    print(f"Probability it's a photo of sasuke uchiha: {probs[1]}")